<a href="https://colab.research.google.com/github/Utomi-Tom/Granular_Material_analysis/blob/main/Geomaterial_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implentation of tools 

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load data

In [4]:
data = "/content/drive/MyDrive/Analoguematerial_/Material_test.csv"
dataset = pd.read_csv(data,)

## DATA QC AND QA

In [10]:
# Rename the parameters with undersandable names.
Cols = ["Material","Normal_stress_Pa","Shear_stress_Pa","Tensile_stress_Pa", "Cohesion_Pa", "UCS_Pa", "Sigma1_Pa", "Sigma2_Pa", 
        "Effective_Angle_Friction", "React_Angle_Friction", "Density_g/cm³", "Flowability"]

Dataset = dataset.set_axis(Cols, axis="columns")
Dataset.sample(frac= 0.2)

,Material,Normal_stress_Pa,Shear_stress_Pa,Tensile_stress_Pa,Cohesion_Pa,UCS_Pa,Sigma1_Pa,Sigma2_Pa,Effective_Angle_Friction,React_Angle_Friction,Density_g/cm³,Flowability
2,GP 100,587,552,226,193,662,1484,281,41.8,37.3,0.659,2.24
30,flour 100,1074,797,530,421,1200,2230,506,37.0,26.5,0.605,1.86
33,flour 100,1566,1088,546,473,1411,3232,817,35.5,20.8,0.623,2.29
6,GP 100,1567,1435,499,445,1588,4070,780,41.8,40.5,0.736,2.56
23,GP25-SN75,1132,859,108,210,727,2633,634,37.4,32.0,1.177,3.62
32,flour 100,1566,1227,163,550,1727,3573,750,39.3,30.0,0.620,2.07
7,GP 100,1574,1451,508,455,1630,4126,779,42.1,41.3,0.731,2.53


In [11]:
# Check for missing values
Dataset.isna().sum()

Material                    0
Normal_stress_Pa            0
Shear_stress_Pa             0
Tensile_stress_Pa           0
Cohesion_Pa                 0
UCS_Pa                      0
Sigma1_Pa                   0
Sigma2_Pa                   0
Effective_Angle_Friction    0
React_Angle_Friction        0
Density_g/cm³               0
Flowability                 0
dtype: int64

# Feature Engineering

In [12]:
# Create a column on the Dataset that qualifies material rheology based on Flowability and cohesion
import numpy as np

Dataset["Rheology"] = np.where(Dataset['Flowability'] < 3.0, "Cohesive", "Incohesive")


# The slip tendency (T,) of a surface is defined as the ratio of shear stress to normal stress on that surface:
Dataset["Slip_Tendency"] = Dataset["Shear_stress_Pa"] / Dataset["Normal_stress_Pa"]

Dataset.sample(7)

,Material,Normal_stress_Pa,Shear_stress_Pa,Tensile_stress_Pa,Cohesion_Pa,UCS_Pa,Sigma1_Pa,Sigma2_Pa,Effective_Angle_Friction,React_Angle_Friction,Density_g/cm³,Flowability,Rheology,Slip_Tendency
13,GP75-SN25,1587,1275,452,422,1403,3686,827,38.5,37.4,0.864,2.63,Cohesive,0.803403
3,GP 100,1077,889,290,378,1212,2531,512,40.1,37.1,0.707,2.09,Cohesive,0.825441
16,GP50-SN50,1106,889,347,345,1109,2533,552,38.8,35.4,0.949,2.28,Cohesive,0.803797
33,flour 100,1566,1088,546,473,1411,3232,817,35.5,20.8,0.623,2.29,Cohesive,0.694764
30,flour 100,1074,797,530,421,1200,2230,506,37.0,26.5,0.605,1.86,Cohesive,0.742086
32,flour 100,1566,1227,163,550,1727,3573,750,39.3,30.0,0.620,2.07,Cohesive,0.783525
29,flour 100,1074,808,293,462,1289,2226,482,37.6,26.9,0.598,1.73,Cohesive,0.752328


In [ ]:
# Create independent variables and target column for training . 
X = Shuff.drop(['Material',"Rheology"], axis = 1)
Y = Shuff["Rheology"]

In [ ]:
# split dataset 
from sklearn.model_selection import train_test_split

x_tr, x_t, y_tr, y_t = train_test_split(X,Y,
                                        test_size=0.3, )